In [76]:
import gmsh
gmsh.initialize()
using LinearAlgebra, SparseArrays

In [77]:
# Adatok
height = 1000
base = 1000
elemSize = 10
traction = 1

approxOrder = 3
internalNodes = 1
quadElements = 1

1

In [78]:
# Geometria megrajzolása
gmsh.model.add("rectangle")

p1 = gmsh.model.occ.addPoint(0, 0, 0)
p2 = gmsh.model.occ.addPoint(base, 0, 0)
p3 = gmsh.model.occ.addPoint(base, height, 0)
p4 = gmsh.model.occ.addPoint(0, height, 0)

l1 = gmsh.model.occ.addLine(p1, p2)
l2 = gmsh.model.occ.addLine(p2, p3)
l3 = gmsh.model.occ.addLine(p3, p4)
l4 = gmsh.model.occ.addLine(p4, p1)

cl1 = gmsh.model.occ.addCurveLoop([l1, l2, l3, l4])

sf1 = gmsh.model.occ.addPlaneSurface([cl1])

gmsh.model.occ.synchronize()

In [79]:
# Megfogások és terhelések helyének megadása
# az egyes peremfeltételek helyét megadó geometriai objektumokat
# csoportokba rendezzük

# megfogás 1
supp1 = gmsh.model.addPhysicalGroup(1, [l1])
gmsh.model.setPhysicalName(1, supp1, "FixBottomY")

# megfogás 2
supp2 = gmsh.model.addPhysicalGroup(1, [l4])
gmsh.model.setPhysicalName(1, supp2, "FixLeftX")

# terhelés
trac1 = gmsh.model.addPhysicalGroup(1, [l3])
gmsh.model.setPhysicalName(1, trac1, "ForceTopY")

In [80]:
# Végeselemes felosztás elvégzése

# lekérjük az összes csomópontot
all = gmsh.model.getEntities(0)
# megadjuk, hogy a csomóponthoz rendelt eleméret mekkora legyen
gmsh.model.mesh.setSize(all, elemSize)
# kiválasztjuk a 8-as számú hálózó algoritmust a 2D-s sf1 felülethez
gmsh.model.mesh.setAlgorithm(2, sf1, 8)
# legeneráljuk a hálót a felület kontúrjához (1D-s)
gmsh.model.mesh.generate(1)
# legeneráljuk a hálót a felülethez (2D-s)
gmsh.model.mesh.generate(2)
# a legenerált háromszög elemeket négyszög elemekké alakítjuk
if quadElements == 1
    gmsh.model.mesh.recombine()
end
# másodfokú elemekhez:
# belső csomópontok használata
if internalNodes == 1
    gmsh.option.setNumber("Mesh.SecondOrderIncomplete", 0) # 0:belső csomópontokkal 1:csak éleken lévő csomópontokkal
else
    gmsh.option.setNumber("Mesh.SecondOrderIncomplete", 1) # 0:belső csomópontokkal 1:csak éleken lévő csomópontokkal
end
# közelítés fokszáma (1-től 5-ig)
gmsh.model.mesh.setOrder(approxOrder)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000508879s, CPU 0.000511s)
Info    : 400 nodes 404 elements
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay for Quads)
Info    : Done meshing 2D (Wall 1.28753s, CPU 1.24473s)
Info    : 10202 nodes 20406 elements
Info    : Recombining 2D mesh...
Info    : Blossom: 29803 internal 396 closed
Info    : Blossom recombination completed (Wall 0.328099s, CPU 0.327976s): 10000 quads, 0 triangles, 0 invalid quads, 0 quads with Q < 0.1, avg Q = 0.999484, min Q = 0.974711
Info    : Done recombining 2D mesh (Wall 0.330148s, CPU 0.330015s)
Info    : Meshing order 3 (curvilinear on)...
Info    : [  0%] Meshing curve 1 order 3
Info    : [ 20%] Meshing curve 2 order 3
Info    : [ 40%] Meshing curve 3 order 3
Info    : [ 60%] Meshing curve 4 order 3
Info    : [

In [81]:
# Néhány láthatósági beállítás 0:kikapcs. 1:bekapcs.
gmsh.option.setNumber("Geometry.CurveLabels", 0)
gmsh.option.setNumber("Mesh.Points", 0)
gmsh.option.setNumber("Mesh.Lines", 0)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("Mesh.NodeLabels", 0)
gmsh.option.setNumber("Mesh.LineLabels", 0)
gmsh.option.setNumber("Mesh.SurfaceLabels", 0)

In [82]:
# Anyagállandók mátrixa SA feladathoz
E = 2e5
ν = 0.3
D = E / ((1 + ν) * (1 - 2ν)) * [1-ν ν 0; ν 1-ν 0; 0 0 (1-2ν)/2]

3×3 Matrix{Float64}:
 2.69231e5  1.15385e5      0.0
 1.15385e5  2.69231e5      0.0
 0.0        0.0        76923.1

In [83]:
# Anyagállandók mátrixa ÁSF feladathoz
E = 2e5
ν = 0.3
D = E / (1 - ν^2) * [1 ν 0; ν 1 0; 0 0 (1-ν)/2]

3×3 Matrix{Float64}:
     2.1978e5  65934.1           0.0
 65934.1           2.1978e5      0.0
     0.0           0.0       76923.1

In [84]:
a = [[1, 2, 3, 7], [4, 5, 6, 3, 5], [7, 8]]
sum([length(a[i]) for i in 1:length(a)])

11

In [85]:
# Merevségi mátrix felépítése
# csomópontok sorszámának lekérése
nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
# végeselemek típusának, sorszámának és kapcsolati mátrixának (connectivity matrix) lekérése
elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(2, -1)
# a lefoglalandó memória méretének becslése
lengthOfIJV = sum([(div(length(elemNodeTags[i]), length(elemTags[i])) * 2)^2 * length(elemTags[i]) for i in 1:length(elemTags)])
nn = []
I = []
J = []
V = []
V = convert(Vector{Float64}, V)
sizehint!(I, lengthOfIJV)
sizehint!(J, lengthOfIJV)
sizehint!(V, lengthOfIJV)
for i in 1:length(elemTypes)
    et = elemTypes[i]
    elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
    intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2approxOrder))
    numIntPoints = length(intWeights)
    comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
    ∇h = reshape(dfun, :, numIntPoints)
    nnet = zeros(Int, length(elemTags[i]), numNodes)
    invJac = zeros(3, 3numIntPoints)
    Iidx = zeros(Int, numNodes * 2, numNodes * 2)
    Jidx = zeros(Int, numNodes * 2, numNodes * 2)
    for k in 1:numNodes*2, l in 1:numNodes*2
        Iidx[k, l] = l
        Jidx[k, l] = k
    end
    ∂h = zeros(3, numNodes * numIntPoints) # ∂h-t mindig csak felül kellene írni, nem kell újra és újra memóriát foglalni neki.
    B = zeros(3 * numIntPoints, 2 * numNodes) # B-t mindig csak felül kellene írni?
    K1 = zeros(2 * numNodes, 2 * numNodes)
    nn2 = zeros(Int, 2 * numNodes)
    for j in 1:length(elemTags[i])
        elem = elemTags[i][j]
        for k in 1:numNodes
            nnet[j, k] = elemNodeTags[i][(j-1)*numNodes+k]
        end
        jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
        Jac = reshape(jac, 3, :)
        for k in 1:numIntPoints
            invJac[1:3, 3*k-2:3*k] = inv(Jac[1:3, 3*k-2:3*k])'
        end
        #∂h = zeros(3, numNodes * numIntPoints) # ∂h-t mindig csak felül kellene írni, nem kell újra és újra memóriát foglalni neki.
        ∂h *= 0
        for k in 1:numIntPoints, l in 1:numNodes
            ∂h[1:2, (k-1)*numNodes+l] = invJac[1:2, k*3-2:k*3-1] * ∇h[l*3-2:l*3-1, k] #??????????????????
        end
        #B = zeros(3 * numIntPoints, 2 * numNodes) # B-t mindig csak felül kellene írni?
        B *= 0
        for k in 1:numIntPoints, l in 1:numNodes
            B[k*3-0, l*2-0] = B[k*3-2, l*2-1] = ∂h[1, (k-1)*numNodes+l]
            B[k*3-0, l*2-1] = B[k*3-1, l*2-0] = ∂h[2, (k-1)*numNodes+l]
        end
        #K1 = zeros(2 * numNodes, 2 * numNodes)
        K1 *= 0
        for k in 1:numIntPoints
            B1 = B[k*3-2:k*3, 1:2*numNodes]
            K1 += B1' * D * B1 * jacDet[k] * intWeights[k]
        end
        nn2[1:2:2*numNodes] = 2 * nnet[j, 1:numNodes] .- 1
        nn2[2:2:2*numNodes] = 2 * nnet[j, 1:numNodes]
        append!(I, nn2[Iidx[:]])
        append!(J, nn2[Jidx[:]])
        append!(V, K1[:])
    end
    push!(nn, nnet)
end
K = sparse(I, J, V)
K0 = copy(K)
display(K)

181202×181202 SparseMatrixCSC{Float64, Int64} with 9012004 stored entries:
⎡⠻⣦⡀⠀⠀⣄⣶⣆⠀⢸⡼⠷⠶⢶⣶⣦⣤⣤⣤⣄⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⢠⡀⢠⡀⠀⠀⠀⠀⠀⢠⎤
⎢⠀⠈⠻⣦⡀⣿⣧⠹⣧⠈⣇⠀⠀⠀⠀⠀⠀⠉⠉⠉⠛⠛⠻⠷⢶⣦⣄⣀⡀⠀⠈⢻⡌⠻⠶⣤⣄⡀⣦⣼⎥
⎢⠀⢤⣤⣬⣿⣿⣿⣤⣼⣷⣽⠀⠀⠀⠀⠠⠤⠤⠠⠄⢤⢤⣄⣀⣀⣤⣬⣭⣽⣿⣶⣠⣿⣀⣤⣤⣬⣿⡏⣿⎥
⎢⠸⢿⣍⡛⠛⣿⡿⣯⡛⠻⡝⠺⠿⠲⠶⠦⠤⠬⢬⣭⣭⣉⣉⣉⡉⠛⠛⠛⠛⠛⢻⣙⠛⣉⡙⠛⠙⠋⠛⢻⎥
⎢⣀⣀⡉⠛⢶⣿⣿⡈⠻⣦⣳⣀⣀⣀⣀⣀⣀⣀⣀⠀⠀⠀⠉⠉⠉⠉⠓⠒⠒⠶⣄⠘⠷⠈⠉⠙⠓⠲⡟⢓⎥
⎢⢶⡏⠉⠙⠓⠛⣳⡉⠙⢺⢻⣶⣶⣆⠀⠁⠈⠉⠉⠉⠉⠉⠉⠉⠉⠉⠛⠓⠒⠒⠚⠉⠓⠉⠁⠉⠙⠒⠁⠘⎥
⎢⢸⣇⠀⠀⠀⠀⢻⡃⠀⢸⠸⢿⣿⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⎥
⎢⠸⣿⠀⠀⠀⡀⠸⡇⠀⢸⠄⠀⠻⣿⣿⣿⣷⣖⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⠀⠀⠀⠀⠀⠀⠀⠀⢀⎥
⎢⠀⣿⡄⠀⠀⡇⡀⡇⠀⢸⡆⠀⠀⠈⢹⢿⣿⣿⣷⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⠆⢸⠀⠀⠀⠀⠀⠀⢸⎥
⎢⠀⢿⡇⠀⠀⠆⡆⣷⠀⠘⡇⠀⠀⠀⠀⠉⠙⣿⣿⣿⣷⡆⠀⠀⠀⠀⠀⠀⠀⠀⢸⡇⢸⡀⠀⠀⠀⠀⠀⠰⎥
⎢⠀⠸⣿⠀⠀⣗⡇⢻⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠹⠿⣿⣿⣧⡀⠀⠀⠀⠀⠀⠀⠘⣇⠈⣧⠀⠀⠀⠀⠀⢸⎥
⎢⠀⠀⢿⡆⠀⢹⡇⢸⡇⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠻⣿⣿⣆⡀⠀⠀⠀⠀⠀⢿⠀⢹⣇⠀⠀⠀⠀⠈⎥
⎢⠀⠀⠸⣷⠀⣼⣧⠈⡇⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠹⢿⣷⡄⠀⠀⠀⠀⢸⡀⠀⠹⣦⡄⠀⣇⣢⎥
⎢⠀⠀⠀⢹⡆⣿⣿⠀⢹⠀⢿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⢿⣷⣄⠀⠀⠘⡇⠀⠀⠘⢿⡀⡹⣸⎥
⎢⠀⠀⠀⠈⣷⣿⣿⠀⢸⡄⢸⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠻⣦⡀⠀⣷⠀⠀⠀⠀⢷⡀⢳⎥
⎢⠀⠲⣦⣀⠘⣻⣟⢲⣀⠙⡞⠀⠀⠀⠒⠒⠲⠖⠶⠶⠶⢤⣤⣄⣀⣀⣀⠀⠀⠈⠻⣦⡸⣦⣄⣀⡀⠀⢃⢙⎥
⎢⠀⠲⣦⡉⠛⢻⡟⢠⡙⠃⡝⠀⠀⠀⠀⠀⠒⠒⠒⠲⠦⣤⣄⣀⠀⠈⠉⠉⠙⠛⠲⣮⠻⣦⡀⠉⠉⠛⡋⠙⎥
⎢⠀⠀⠘⣧⠀⣿⣷⠈⣇⠀⡅⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠙⠳⣦⣀⠀⠀⠀⠀⢹⡄⠈⠻⣦⡄⠀⣧⠠⎥
⎢⠀⠀⠀⠹⣦⣿⡷⠀⢹⡀⢳⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠛⠳⢤⣄⠀⠈⣧⠀⠀⠉⠻⣦⡿⣶⎥
⎣⠀⣀⣈⣿⣯⣭⣿⣀⢿⢉⣁⠀⠀⢀⠀⢀⣀⣀⢀⡀⣀⣀⡀⠀⠩⣹⣓⣪⢤⣈⣍⢐⣏⠈⠉⡛⢻⣯⠿⣧⎦

In [86]:
# Teher vektor felépítése
p = [0, 1]
dof, dof = size(K)
f = zeros(dof)
tags = gmsh.model.getEntitiesForPhysicalGroup(1, trac1)
for i ∈ 1:length(tags)
    elementTypes, elementTags, elemNodeTags = gmsh.model.mesh.getElements(1, tags[i])
    if length(elementTypes) != 1
        error("A peremen nem csak egyfajta végeselem van!")
    end
    #elements = elementTags[1]
    elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(elementTypes[1])
    nnoe = reshape(elemNodeTags[1], numNodes, :)'
    intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(elementTypes[1], "Gauss" * string(approxOrder))
    numIntPoints = length(intWeights)
    comp, fun, ori = gmsh.model.mesh.getBasisFunctions(elementTypes[1], intPoints, "Lagrange")
    h = reshape(fun, :, numIntPoints)
    comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(elementTypes[1], intPoints, "GradLagrange")
    ∇h = reshape(dfun, :, numIntPoints)
    H = zeros(2 * numIntPoints, 2 * numNodes)
    for j in 1:numIntPoints
        for k in 1:numNodes
            H[j*2-1, k*2-1] = h[k, j]
            H[j*2-0, k*2-0] = h[k, j]
        end
    end
    f1 = zeros(2 * numNodes)
    nn2 = zeros(Int, 2 * numNodes)
    for l in 1:length(elementTags[1])
        elem = elementTags[1][l]
        jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
        Jac = reshape(jac, 3, :)
        f1 *= 0
        for j in 1:numIntPoints
            H1 = H[j*2-1:j*2, 1:2*numNodes]
            Ja = √((Jac[1, 3*j-2])^2 + (Jac[2, 3*j-2])^2)
            f1 += H1' * p * Ja * intWeights[j]
        end
        nn2[1:2:2*numNodes] = 2 * nnoe[l, 1:numNodes] .- 1
        nn2[2:2:2*numNodes] = 2 * nnoe[l, 1:numNodes]
        f[nn2] += f1
    end
end
display(f)

181202-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 1.2500000000000324
 0.0
 1.2499999999999898
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [87]:
nodeTags, coord = gmsh.model.mesh.getNodesForPhysicalGroup(1, supp1)
nodeTags *= 2
for i ∈ nodeTags
    K[i, :] .= 0
    K[:, i] .= 0
    K[i, i] = 1
    f[i] = 0
end
dropzeros!(K)

181202×181202 SparseMatrixCSC{Float64, Int64} with 8989761 stored entries:
⎡⠻⣦⡀⠀⠀⣄⣶⣆⠀⢸⡼⠷⠶⢶⣶⣦⣤⣤⣤⣄⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⢠⡀⢠⡀⠀⠀⠀⠀⠀⢠⎤
⎢⠀⠈⠻⣦⡀⣿⣧⠹⣧⠈⣇⠀⠀⠀⠀⠀⠀⠉⠉⠉⠛⠛⠻⠷⢶⣦⣄⣀⡀⠀⠈⢻⡌⠻⠶⣤⣄⡀⣦⣼⎥
⎢⠀⢤⣤⣬⣿⣿⣿⣤⣼⣷⣽⠀⠀⠀⠀⠠⠤⠤⠠⠄⢤⢤⣄⣀⣀⣤⣬⣭⣽⣿⣶⣠⣿⣀⣤⣤⣬⣿⡏⣿⎥
⎢⠸⢿⣍⡛⠛⣿⡿⣯⡛⠻⡝⠺⠿⠲⠶⠦⠤⠬⢬⣭⣭⣉⣉⣉⡉⠛⠛⠛⠛⠛⢻⣙⠛⣉⡙⠛⠙⠋⠛⢻⎥
⎢⣀⣀⡉⠛⢶⣿⣿⡈⠻⣦⣳⣀⣀⣀⣀⣀⣀⣀⣀⠀⠀⠀⠉⠉⠉⠉⠓⠒⠒⠶⣄⠘⠷⠈⠉⠙⠓⠲⡟⢓⎥
⎢⢶⡏⠉⠙⠓⠛⣳⡉⠙⢺⢻⣶⣶⣆⠀⠁⠈⠉⠉⠉⠉⠉⠉⠉⠉⠉⠛⠓⠒⠒⠚⠉⠓⠉⠁⠉⠙⠒⠁⠘⎥
⎢⢸⣇⠀⠀⠀⠀⢻⡃⠀⢸⠸⢿⣿⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⎥
⎢⠸⣿⠀⠀⠀⡀⠸⡇⠀⢸⠄⠀⠻⣿⣿⣿⣷⣖⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⠀⠀⠀⠀⠀⠀⠀⠀⢀⎥
⎢⠀⣿⡄⠀⠀⡇⡀⡇⠀⢸⡆⠀⠀⠈⢹⢿⣿⣿⣷⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⠆⢸⠀⠀⠀⠀⠀⠀⢸⎥
⎢⠀⢿⡇⠀⠀⠆⡆⣷⠀⠘⡇⠀⠀⠀⠀⠉⠙⣿⣿⣿⣷⡆⠀⠀⠀⠀⠀⠀⠀⠀⢸⡇⢸⡀⠀⠀⠀⠀⠀⠰⎥
⎢⠀⠸⣿⠀⠀⣗⡇⢻⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠹⠿⣿⣿⣧⡀⠀⠀⠀⠀⠀⠀⠘⣇⠈⣧⠀⠀⠀⠀⠀⢸⎥
⎢⠀⠀⢿⡆⠀⢹⡇⢸⡇⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠻⣿⣿⣆⡀⠀⠀⠀⠀⠀⢿⠀⢹⣇⠀⠀⠀⠀⠈⎥
⎢⠀⠀⠸⣷⠀⣼⣧⠈⡇⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠹⢿⣷⡄⠀⠀⠀⠀⢸⡀⠀⠹⣦⡄⠀⣇⣢⎥
⎢⠀⠀⠀⢹⡆⣿⣿⠀⢹⠀⢿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⢿⣷⣄⠀⠀⠘⡇⠀⠀⠘⢿⡀⡹⣸⎥
⎢⠀⠀⠀⠈⣷⣿⣿⠀⢸⡄⢸⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠻⣦⡀⠀⣷⠀⠀⠀⠀⢷⡀⢳⎥
⎢⠀⠲⣦⣀⠘⣻⣟⢲⣀⠙⡞⠀⠀⠀⠒⠒⠲⠖⠶⠶⠶⢤⣤⣄⣀⣀⣀⠀⠀⠈⠻⣦⡸⣦⣄⣀⡀⠀⢃⢙⎥
⎢⠀⠲⣦⡉⠛⢻⡟⢠⡙⠃⡝⠀⠀⠀⠀⠀⠒⠒⠒⠲⠦⣤⣄⣀⠀⠈⠉⠉⠙⠛⠲⣮⠻⣦⡀⠉⠉⠛⡋⠙⎥
⎢⠀⠀⠘⣧⠀⣿⣷⠈⣇⠀⡅⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠙⠳⣦⣀⠀⠀⠀⠀⢹⡄⠈⠻⣦⡄⠀⣧⠠⎥
⎢⠀⠀⠀⠹⣦⣿⡷⠀⢹⡀⢳⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠛⠳⢤⣄⠀⠈⣧⠀⠀⠉⠻⣦⡿⣶⎥
⎣⠀⣀⣈⣿⣯⣭⣿⣀⢿⢉⣁⠀⠀⢀⠀⢀⣀⣀⢀⡀⣀⣀⡀⠀⠩⣹⣓⣪⢤⣈⣍⢐⣏⠈⠉⡛⢻⣯⠿⣧⎦

In [88]:
nodeTags, coord = gmsh.model.mesh.getNodesForPhysicalGroup(1, supp2)
nodeTags *= 2
nodeTags .-= 1
for i ∈ nodeTags
    K[i, :] .= 0
    K[:, i] .= 0
    K[i, i] = 1
    f[i] = 0
end
dropzeros!(K)

181202×181202 SparseMatrixCSC{Float64, Int64} with 8967579 stored entries:
⎡⠻⣦⡀⠀⠀⣄⣶⣆⠀⢸⡼⠷⠶⢶⣶⣦⣤⣤⣤⣄⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⢠⡀⢠⡀⠀⠀⠀⠀⠀⢠⎤
⎢⠀⠈⠻⣦⡀⣿⣧⠹⣧⠈⣇⠀⠀⠀⠀⠀⠀⠉⠉⠉⠛⠛⠻⠷⢶⣦⣄⣀⡀⠀⠈⢻⡌⠻⠶⣤⣄⡀⣦⣼⎥
⎢⠀⢤⣤⣬⣿⣿⣿⣤⣼⣷⣽⠀⠀⠀⠀⠠⠤⠤⠠⠄⢤⢤⣄⣀⣀⣤⣬⣭⣽⣿⣶⣠⣿⣀⣤⣤⣬⣿⡏⣿⎥
⎢⠸⢿⣍⡛⠛⣿⡿⣯⡛⠻⡝⠺⠿⠲⠶⠦⠤⠬⢬⣭⣭⣉⣉⣉⡉⠛⠛⠛⠛⠛⢻⣙⠛⣉⡙⠛⠙⠋⠛⢻⎥
⎢⣀⣀⡉⠛⢶⣿⣿⡈⠻⣦⣳⣀⣀⣀⣀⣀⣀⣀⣀⠀⠀⠀⠉⠉⠉⠉⠓⠒⠒⠶⣄⠘⠷⠈⠉⠙⠓⠲⡟⢓⎥
⎢⢶⡏⠉⠙⠓⠛⣳⡉⠙⢺⢻⣶⣶⣆⠀⠁⠈⠉⠉⠉⠉⠉⠉⠉⠉⠉⠛⠓⠒⠒⠚⠉⠓⠉⠁⠉⠙⠒⠁⠘⎥
⎢⢸⣇⠀⠀⠀⠀⢻⡃⠀⢸⠸⢿⣿⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⎥
⎢⠸⣿⠀⠀⠀⡀⠸⡇⠀⢸⠄⠀⠻⣿⣿⣿⣷⣖⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⠀⠀⠀⠀⠀⠀⠀⠀⢀⎥
⎢⠀⣿⡄⠀⠀⡇⡀⡇⠀⢸⡆⠀⠀⠈⢹⢿⣿⣿⣷⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⠆⢸⠀⠀⠀⠀⠀⠀⢸⎥
⎢⠀⢿⡇⠀⠀⠆⡆⣷⠀⠘⡇⠀⠀⠀⠀⠉⠙⣿⣿⣿⣷⡆⠀⠀⠀⠀⠀⠀⠀⠀⢸⡇⢸⡀⠀⠀⠀⠀⠀⠰⎥
⎢⠀⠸⣿⠀⠀⣗⡇⢻⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠹⠿⣿⣿⣧⡀⠀⠀⠀⠀⠀⠀⠘⣇⠈⣧⠀⠀⠀⠀⠀⢸⎥
⎢⠀⠀⢿⡆⠀⢹⡇⢸⡇⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠻⣿⣿⣆⡀⠀⠀⠀⠀⠀⢿⠀⢹⣇⠀⠀⠀⠀⠈⎥
⎢⠀⠀⠸⣷⠀⣼⣧⠈⡇⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠹⢿⣷⡄⠀⠀⠀⠀⢸⡀⠀⠹⣦⡄⠀⣇⣢⎥
⎢⠀⠀⠀⢹⡆⣿⣿⠀⢹⠀⢿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⢿⣷⣄⠀⠀⠘⡇⠀⠀⠘⢿⡀⡹⣸⎥
⎢⠀⠀⠀⠈⣷⣿⣿⠀⢸⡄⢸⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠻⣦⡀⠀⣷⠀⠀⠀⠀⢷⡀⢳⎥
⎢⠀⠲⣦⣀⠘⣻⣟⢲⣀⠙⡞⠀⠀⠀⠒⠒⠲⠖⠶⠶⠶⢤⣤⣄⣀⣀⣀⠀⠀⠈⠻⣦⡸⣦⣄⣀⡀⠀⢃⢙⎥
⎢⠀⠲⣦⡉⠛⢻⡟⢠⡙⠃⡝⠀⠀⠀⠀⠀⠒⠒⠒⠲⠦⣤⣄⣀⠀⠈⠉⠉⠙⠛⠲⣮⠻⣦⡀⠉⠉⠛⡋⠙⎥
⎢⠀⠀⠘⣧⠀⣿⣷⠈⣇⠀⡅⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠙⠳⣦⣀⠀⠀⠀⠀⢹⡄⠈⠻⣦⡄⠀⣧⠠⎥
⎢⠀⠀⠀⠹⣦⣿⡷⠀⢹⡀⢳⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠛⠳⢤⣄⠀⠈⣧⠀⠀⠉⠻⣦⡿⣶⎥
⎣⠀⣀⣈⣿⣯⣭⣿⣀⢿⢉⣁⠀⠀⢀⠀⢀⣀⣀⢀⡀⣀⣀⡀⠀⠩⣹⣓⣪⢤⣈⣍⢐⣏⠈⠉⡛⢻⣯⠿⣧⎦

In [89]:
q = K \ f

181202-element Vector{Float64}:
  0.0
  0.0
 -0.0014999999999941074
  0.0
 -0.0014999999999963348
  0.004999999999993549
  0.0
  0.004999999999990859
 -1.49999999999199e-5
  0.0
  ⋮
  0.0024678431499649393
 -0.0007599959873130867
  0.002537660896019818
 -0.0007550260258546747
  0.002537596867717633
 -0.0007550601267149788
  0.0025211420427589224
 -0.0007600378256493482
  0.0025212741774846698

In [90]:
p[2] / (E / height)

0.005

In [91]:
-(p[2] / E) * ν * base

-0.0015

In [92]:
K0 * q

181202-element Vector{Float64}:
 -1.156408302449563e-12
 -1.2499999999971871
  1.5432100042289676e-14
 -1.2499999999988756
 -2.842170943040401e-14
  1.2499999999999005
 -1.1368683772161603e-12
  1.2500000000001705
  1.3322676295501878e-15
 -2.499999999994368
  ⋮
  0.0
 -1.580957587066223e-13
  6.821210263296962e-13
  0.0
  1.1368683772161603e-13
  4.707345624410664e-14
 -1.1368683772161603e-12
 -2.333688797762079e-13
  0.0

In [93]:
k = base * E / height

200000.0

In [94]:
gmsh.option.setNumber("View.IntervalsType", 3)
gmsh.option.setNumber("View.VectorType", 5)

In [95]:
nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
non = length(nodeTags)
uvec = gmsh.view.add("uvec")
u = zeros(3 * non)
for i in 1:length(nodeTags)
    u[3i-2] = q[2*nodeTags[i]-1]
    u[3i-1] = q[2*nodeTags[i]-0]
end
gmsh.view.addHomogeneousModelData(uvec, 0, "rectangle", "NodeData", nodeTags, u, 0, 3)


In [96]:
nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
non = length(nodeTags)
ux = gmsh.view.add("ux")
u = zeros(non)
for i in 1:length(nodeTags)
    u[i] = q[2*nodeTags[i]-1]
    #u[3i-1] = q[2*nodeTags[i]-0]
end
gmsh.view.addHomogeneousModelData(ux, 0, "rectangle", "NodeData", nodeTags, u, 0, 1)


In [97]:
nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
non = length(nodeTags)
uy = gmsh.view.add("uy")
u = zeros(non)
for i in 1:length(nodeTags)
    #u[3i-2] = q[2*nodeTags[i]-1]
    u[i] = q[2*nodeTags[i]-0]
end
gmsh.view.addHomogeneousModelData(uy, 0, "rectangle", "NodeData", nodeTags, u, 0, 1)


In [98]:
#gmsh.write("rect.geo")
#gmsh.write("rect.msh")
#gmsh.write("rect.pos")

In [99]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20221221
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.6.1
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [100]:
gmsh.finalize()